## comments
position encoding -> 
    # if train_len is None:
    #     pos_list = np.arange(seq_len)
    # else:
    #     pos_list = np.linspace(0, train_len-1, num=seq_len)
    pos_list = np.arange(seq_len)

encoding layer -> 2 layers instead of 3

The model failed to align BECAUSE not training enough, check transformer_l2 for more

In [1]:
import sys
sys.path.append("/home/c1l1mo/testings/carl")
sys.path.append("/home/c1l1mo/testings/carl/utils")
sys.path.append("/home/c1l1mo/testings/carl/datasets")
print(sys.path)

['/home/c1l1mo/projects/VideoAlignment', '/home/c1l1mo/miniconda3/envs/carl/lib/python37.zip', '/home/c1l1mo/miniconda3/envs/carl/lib/python3.7', '/home/c1l1mo/miniconda3/envs/carl/lib/python3.7/lib-dynload', '', '/home/c1l1mo/.local/lib/python3.7/site-packages', '/home/c1l1mo/testings/HIT', '/home/c1l1mo/miniconda3/envs/carl/lib/python3.7/site-packages', '/home/c1l1mo/.local/lib/python3.7/site-packages/IPython/extensions', '/tmp/tmpxnjy4i96', '/home/c1l1mo/testings/carl', '/home/c1l1mo/testings/carl/utils', '/home/c1l1mo/testings/carl/datasets']


In [2]:
from model.transformer.transformer import CARL as Mine_Transformer
from models.transformer import TransformerModel as CARL_Transformer

/home/c1l1mo/.local/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import yaml
from easydict import EasyDict as edict
with open("/home/c1l1mo/tmp/scl_transformer_processed_axel_trimmed/config.yml","r") as file:
    cfg = yaml.safe_load(file)
cfg = edict(cfg)

In [4]:
import numpy as np
import torch
import random

def setup_seed(seed):
    random.seed(seed)                          
    np.random.seed(seed)                       
    torch.manual_seed(seed)                    
    torch.cuda.manual_seed(seed)               
    torch.cuda.manual_seed_all(seed)           
    torch.backends.cudnn.deterministic = True  


setup_seed(7)

cfg.MODEL.PROJECTION = False
cfg.MODEL.L2_NORMALIZE= False

mine = Mine_Transformer()
carl = CARL_Transformer(cfg)

mine.transformation.fc_layer[1].weight.data.fill_(0.01)
mine.transformation.fc_layer[1].bias.data.fill_(0.01)
mine.transformation.fc_layer[5].weight.data.fill_(0.01)
mine.transformation.fc_layer[5].bias.data.fill_(0.01)
mine.transformation.video_emb.weight.data.fill_(0.01)
mine.transformation.video_emb.bias.data.fill_(0.01)
for name,parameter in mine.encoder.encodingLayers.named_parameters():
    parameter.data.fill_(0.01)
mine.encoder.embedding_layer.weight.data.fill_(0.01)
mine.encoder.embedding_layer.bias.data.fill_(0.01)

carl.embed.fc_layers[1].weight.data.fill_(0.01)
carl.embed.fc_layers[1].bias.data.fill_(0.01)
carl.embed.fc_layers[5].weight.data.fill_(0.01)
carl.embed.fc_layers[5].bias.data.fill_(0.01)
carl.embed.video_emb.weight.data.fill_(0.01)
carl.embed.video_emb.bias.data.fill_(0.01)
for name,parameter in carl.embed.video_encoder.named_parameters():
    parameter.data.fill_(0.01)
carl.embed.embedding_layer.weight.data.fill_(0.01)
carl.embed.embedding_layer.bias.data.fill_(0.01)


/home/c1l1mo/.local/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/c1l1mo/.local/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 

In [5]:
test = torch.rand(1,14,3,224,224)
test.shape

torch.Size([1, 14, 3, 224, 224])

In [6]:
mine.eval()
m = mine(test)

In [7]:
carl.eval()
c = carl(test)

In [8]:
for m_,c_ in zip(m,c):
    print(m_,c_)
    # print(m[m_].shape,c[c_].shape)
    print(torch.equal(m[m_],c[c_]))

x_res x_res
True
x_reshape x_reshape
True
x_pooling x_pooling
True
x_flatten x_flatten
True
x_fc x_fc
True
x_emb x_emb
True
x_transform x_transform
True
x_pos x_pos
True
x_encoding_layer x_encoding_layer
True
x x
True
